In [8]:
import numpy as np
import pandas as pd
import re
import time

In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [10]:
wd = webdriver.Chrome(ChromeDriverManager().install())
songList = pd.read_csv(
    "../data/lyrics_original/kpop_lyrics_crawling_step1.csv", index_col=False
)




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/June/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
/var/folders/sd/c06408ds1fz2p388rvn68r1w0000gn/T/ipykernel_83831/1397227973.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


In [11]:

songList = songList.loc[songList.year >= 2001]
song_details = pd.DataFrame(columns=['lyric', 'genre'])
detail_base_url = "http://www.melon.com/song/detail.htm?songId="
lyric_tag_regex = re.compile("<.*?>")
song_ids = songList['song_id'].astype(str)

In [5]:
for song_id in song_ids:

    wd.get(detail_base_url + song_id)

    try:
        lyric = wd.find_element(By.CSS_SELECTOR, "div.lyric").get_attribute(
            "textContent"
        )
    except NoSuchElementException:
        lyric = ""
    try:
        genre = (
            wd.find_element(By.CSS_SELECTOR, "div.meta")
            .find_elements(By.CSS_SELECTOR, "dd")[2]
            .get_attribute("textContent")
        )
    except NoSuchElementException:
        pass

    current = pd.DataFrame(
        {
            "song_id": [song_id],
            "lyric": [lyric],
            "genre": [genre],
        }
    )
    song_details = pd.concat((song_details, current))
    time.sleep(np.random.randint(15, 30))


KeyboardInterrupt: 

In [6]:
# song_details
song_details = song_details[:414]

In [7]:
song_details["song_id"] = pd.to_numeric(song_details["song_id"])
song_list = pd.merge(
    songList, song_details, left_on="song_id", right_on="song_id", how="inner"
)
song_list.to_csv(
    "../data/lyrics_original/kpop_lyrics_crawling_step2.csv",
    encoding="utf-8-sig",
    index=False,
)
